In [0]:
%pip install -U 'mlflow>=3.5.0' dspy

In [0]:

import os
import mlflow
import openai
from mlflow.genai.scorers import Correctness, Safety
from mlflow.genai.optimize import GepaPromptOptimizer
from mlflow.genai import datasets

EVAL_DATASET_NAME='<YOUR DATASET NAME>' # Replace with your dataset
dataset = datasets.get_dataset(EVAL_DATASET_NAME)

# Define your prediction function
def predict_fn(**kwargs) -> str:
    prompt = mlflow.genai.load_prompt("prompts:/fabio_goncalves.contract_agent.assistantv1/1")
    completion = openai.OpenAI().chat.completions.create(
        model="gpt-5-mini",  # Replace with your model
        messages=[{"role": "user", "content": prompt.format(**kwargs)}],
    )
    return completion.choices[0].message.content

# Optimize your prompt
result = mlflow.genai.optimize_prompts(
    predict_fn=predict_fn,
    train_data=dataset,
    prompt_uris=["prompts:/fabio_goncalves.contract_agent.assistantv1/1"],
    optimizer=GepaPromptOptimizer(reflection_model="openai:/gpt-5"),
    scorers=[Correctness(), Safety()],
)

# Open the prompt registry page to check the new prompt
print(f"The new prompt URI: {result.optimized_prompts[0].uri}")